# $ \underline{Habilitar \ Warnings:} $

Ahora nos dará más info en los errores, no sale en la versión del compilador 2.13, en la 2.12 si.

In [ ]:
interp.configureCompiler(_.settings.nowarn.value = false)

# $ \underline{·Currificación \ de \ Funciones:} $

La currificación nos permite transformar funciones para que reciban solo un parámetro de entrada. 

In [1]:
def compose[A, B, C]: (B => C, A =>B) => (A => C) = //Recibe 2 parámetros.
    (f2, f1) => (a: A) => f2(f1(a))

def composeCurrified[A, B, C]: (B => C) => ((A => B) => A => C) = //Misma función pero solo recibe uno.
    (f2: B => C) => (f1: (A => B)) => (a: A) => f2(f1(a)): C

defined function compose
defined function composeCurrified

# $ \underline{·Función \ Identidad:} $

Creamos una función (la llamamos identity) que al componerla con otra nos devuelva la segunda función. Además, si componemos otra función con identity nos devuelve la otra función. 

In [ ]:
def identity[A]: A => A = //Definimos la función identidad
    (x: A) => x: A 

compose(isEven: Int => Boolean, identity: Int => Int): (Int => Boolean) //Estamos usando la compose de arriba.
compose(identity: Boolean => Boolean, isEven: Int => Boolean): (Int => Boolean)

compose(isEven, identity[Int]) //Esto es lo mismo q la de arriba
compose(identity[Boolean], isEven) //ya que en la propia compose es suficiente.


# $ \underline{·Azúcar \ Sintáctico:} $

El azúcar sintáctico es un término que busca intentar simplificar la escritura haciendola mínima.

In [ ]:
val addOneV: Int => Int = 
    (a: Int) => a + 1

val addOneV: Int => Int = //Al poner el guión bajo se infiere q será el parámetro q le pasemos.
    _ + 1

val sum: (Int, Int) => Int = 
    (a: Int, b: Int) => a + b

val sum: (Int, Int) => Int = //Hay q tener en cuenta q cada "_" es un párametro distinto,
      _ + _                   // por lo q yo no podría sumar a + a. Ya q entendería el segundo "_" como si fuera otro parámetro sería un a + b.
   



Scala viene incluida con una clase llamada functionx (x es un número cualquiera hasta 22). Esto nos permite aprovecharnos de q ya están implementadas y nos ahorramos la parte de especificar tipos. El número de function (la x) depende de cuantos parámetros de entrada tenga la función.

In [ ]:
val sum: (Int, Int) => Int = 
    (a: Int, b: Int) => a + b

val sum: (Int, Int) => Int = //Misma función pero con azúcar sintáctico.
     _ + _

val sum: Function2[Int, Int, Int] = //Misma función pero con el azúcar de la clase function. Nos saltamos las especificaciones y 2 viene pq tiene
    _ + _                             //2 valores de entrada


La función compose() y isThen (Hace lo contrario a compose) ya vienen en la librería de scala y nos permiten llamarlas así.

In [ ]:
isEven compose length
length isThen isEven

# $ \underline{·Tipos \ Algebraicos \ de \ Datos: }$

Aquí podemos crear valores como productos:

In [ ]:
case class Rectangle(width: Int, heigth: Int) // Creamos un valor de tipo rectángulo a través de un producto de su ancho y alto.
case class Triangle(side: Int)
case class Circle(radius: Double)

type Number = Nothing
type Colour = Nothing
case class Card(number: Number, colour: Colour)

In [ ]:
val r1: Rectangle = Rectangle(1, 3) // Así puedo crear valores tipo rectángulos.
val r2: Rectangle = Rectangle(1, 3)
val r3: Rectangle = Rectangle(2, 4)

r1.width // Estas son observadoras, nos permiten saber q tipos de datos tienen las variables q hemos creado
r1.heigth

r1 == r2 // Esto nos dará verdadero pq los triángulos q creamos en r1 y en r2 tienen las mismas dimensiones.
r2 == r3 // Aquí si q nos dará falso.

### ·Tuplas:

In [ ]:
case class TupleIntInt(fst: Int, snd: Int)
case class TupleNumberColour(fst: number, snd: Colour)

case class Tuple2[A, B](fst: A, snd: B) // Tupla parametrizada
val r1: Tuple2[Int, Int] = Tuple2(1, 3) //Ahora podemos crear un rectángulo solo con esto
val r2: (Int, Int) = (1, 3)

In [ ]:
r1._1 // Así podemos usar los observadores
r1._2

### ·¿Pq se llaman tipos algebraicos?

pq puedo crear tantos valores como el número de posibles valores de uno * el número de posibles valores del otro.

In [ ]:
???: (Boolean, Boolean)
(???: Boolean, ???: Boolean) : (Boolean, Boolean)

//Los 4 posibles valores q pueden tomar:
(true: Boolean, true: Boolean) : (Boolean, Boolean)
(true: Boolean, false: Boolean) : (Boolean, Boolean)
(false: Boolean, false: Boolean) : (Boolean, Boolean)
(false: Boolean, true: Boolean) : (Boolean, Boolean)


### ·Tipo neutro de Datos e Isomorfismo:

In [ ]:
val u: Unit = () // Solo hay un valor, el "()".

Int * Unit // El resultado no es Int, son tipos distintos no los puedo multiplicar. Son isomorfos: puedo establecer una correspondencia uno a
           //uno entre los valores de Int y los de Boolean.

$ Int \neq Unit $ .No son iguales pero son Isomrfos.


In [ ]:
def from(b: Boolean):(Boolean, Unit) =
    (!b: Boolean, (): Unit) //Así asociamos a cada b true una función (false, Unit) y a cada false una (true, Unit).

def to(t2: (Boolean, Unit)): Boolean =
    !t2._1: Boolean //Ojo: usamos los observadores para acceder a los componentes de la función t2, en este caso será true o false.

//Estas son las leyes de este isomorfismo q definimos.
to(from(false)) == false
to(from(true)) == true

from(to((false, ()))) == (false, ())
from(to((true, ()))) == (true, ())

//Otra correspondencia pa el mismo isomorfismo: 
def from(b: Boolean):(Boolean, Unit) =
    (b: Boolean, (): Unit) //Así asociamos a cada b true una función (true, Unit) y a cada false una (false, Unit).

def to(t2: (Boolean, Unit)): Boolean =
    t2._1: Boolean 

A pesar de que Char e Int no son isomorfos (pq hay muchos más Ints q Chars) y no podría establecer una correspondencia uno a uno,
puedo crear un tipo algebraico de datos (en este caso (Char, Int) e (Int, Char)) q sí q lo sea.

In [ ]:
//Esto puedo caer en el exámen.

type A = (Char, Int)
type B = (Int, Char)

def from(t2: (Char, Int) : (Int, Char)) =
    (t2._2: Int, t2._1: Char)

def to(t2: (Int, Char) : (Char, Int)) =
    (t2._2: Char, t2._1: Int)

// Law1:
to(from(("a", 1))) == ("a", 1)
//Law2:
from(to((1, "b"))) == (1, "b")

### $\underline{·Plantilla \ de \ Isomorfismos:}$

In [ ]:
trait Isomorphic[A, B]{
    
    def from(a: A): B
    
    def to(b: B): A
    
    // equality 
    
    def equalA(a1: A, a2: A): Boolean = 
        a1 == a2
    
    def equalB(b1: B, b2: B): Boolean =
        b1 == b2
    
    // Bijection laws
    
    def law1(a: A): Boolean = 
        equalA(to(from(a)), a)
    
    def law2(b: B): Boolean = 
        equalB(from(to(b)), b)
}

## $ \underline{·Valores \ de \ Tipo \ Suma:(Either)} $

Aquí o bien tenemos uno o bien tenemos otro. Ejemplo una variable de carta o bien es sota o bien es caballo o bien es rey. Los valores de tipo producto, por ejemplo un rectángulo, tendrías q tener su base y su altura osea Int * Int. No puedes tener o bien su base o bien su altura, tienes q tener ambos.

Para extraer info de los valores suma hago un pattern matching:


In [ ]:
a match{
    case 1: cosa1 =>
    case 2: cosa2 =>
}

Ahora puedo crear un tipo suma Shape q o bien es círculo o bien es rectángulo o bien es triángulo:

In [ ]:
sealed abstract class Shape

case class Circle(radius: Int) extends Shape
case class Rectangle(width: Int, height: Int) extends Shape
case class Triangle(side: Int) extends Shape

Así puedo declarar valores de tipo Shape:

In [ ]:
val a: Shape = Circle(6)
val r1: Shape = Rectangle(1, 3)

In [ ]:
def toString(s: Shape): String =
    s match{
        case r: Rectangle => "rectangle, width" + r.width + "height" + r.height //Ojo como usamos el pattern matching pa entrar
                                                                                 //al tipo suma y las observadoras ej: r.width 
                                                                                 //pa entrar a los valores de la clase producto
                                                                                 //en este caso Rectangle.
        case c: Circle => "circle" + c.radius
        case t: Triangle => "triangle" + side
    }

In [ ]:
def toString(s: Shape): String = //Versión distinta del pattern matching.
    s match{
        case Rectangle(width, height) => "rectangle, width" + w + "height" + h 
        case Circle(radius) => "circle" + r
        case Triangle(side) => "triangle" + s
    }

### $ \underline{·Valores \ Suma \ Estándar:}$

In [ ]:
object StdSumTypes{
    // type Option[A] = A + 1
    sealed abstract class Option[+A]
    case class Some[A](a: A) extends Option[A]
    case object None extends Option[Nothing]
    
    // Either
    type
}

·Tipo de datos enumerados:

In [ ]:
sealed abstract class Figura
case object Sota extends Figura  //Ojo: aquí no ponemos case class pq son valores fijos, no varían como un rectángulo así q lo pongo como object.
case object Caballo extends Figura
case object Rey extends Figura

### $\underline{·Option:}$

Puedo crear un objeto que permita los fallos, así si se comete alguno me devuelve el nothing en vez de cargarse el programa. Si no falla entra en el Some, si lo hace entra en el None:

In [ ]:
object StdSumTypes{
    // type Option[A] = A + 1
    sealed abstract class Option[+A]
    case class Some[A](a: A) extends Option[A]
    case object None extends Option[Nothing] 
    
    // Either
    sealed abstract class Either[A,B]
    case class Left[A,B](a:A) extends Either[A,B]
    case class Right[A,B](b:B) extends Either[A,B]
}

Esta función fallaría si dividimos entre cero para ello establecemos q si eso pasa devuelvo None y si no devuelvo alguno, osea Some. Para que eso funcione en la salida de la función ponemos un Option[] y dentro lo que nos podría devolver el Some, en este caso un Double:

In [ ]:
def divideWithExceptions(a: Double, b: Double): Option[Double] =
    if (b==0) None
    else Some(a/b)

Lo malo es q el código anterior no nos diría que error hemos cometido así que usamos la clase Either creada arriba. En esta en vez de devolver un Unit puedo devolver un String que si me diga el error que cometí. 

In [ ]:
def divideWithEither(a: Double, b: Double): Either[String, Double] =
    if (b==0) Left("divide by cero":String)
    else Right(a/b)

### $\underline{·Valor \ Neutro \ de \ la \ Suma: \ Tipo \ Nothing}$

In [ ]:
val imposible: Nothing =
    ??? //No podría resolver las interrrogaciones ya que el tipo Nothing no está habitado.

· Isomorfismo de la suma con el tipo nothing.

In [ ]:
class Iso1[A] extends Isomorphism[Either[A, Nothing], A]{

    def from(e: Either[A, Nothing]): A =
        e match{
            case Left(i:A)=> i:A
            //case Right(n: Nothing) => ???: Int //No habría q especificar el caso de la derecha pq para ello n tendría q ser de tipo nothing
                                                  //y no se puede ser de tipo nothing.
        }
    def to(t2:A):Either[A, Nothing] =
        Left(t2:A): Either[A,Nothing]
}


## $ \underline{·Valores \ Tipo \ Exponentes:(Funciones)} $

Son las funciones

In [ ]:
val f:Either[Unit, Either[Unit, Unit]] => Boolean = {
    case Left(u: Unit) =>
    case Right(e: Either[Unit, Unit]) =>
        e match{
            case Left(u: Unit) => ??? :Boolean
            case Right(u: Unit) => ??? :Boolean
        }
}

val f:Either[Unit, Either[Unit, Unit]] => Boolean = {//Tmb puedo escribirlo así q es más compacto. 
    case Left(u: Unit) => ??? :Boolean
    case Right(Left(u: Unit)) => ??? :Boolean   
    case Right(Right(u: Unit)) => ??? :Boolean
}


### $\underline{·Extraer \ Info \ de \ esas \ Funciones:} $

Le aplico esa función a una variable.

$ Ejemplo: \ Z^{X+Y} = Z^{X}·Z^{Y}$

In [ ]:
// Iso extension
class Iso[X,Y,Z] extends Isomorphism[
    Either[X,Y] => Z,
    (X => Z, Y => Z) 
]{
    def from(t2: Either[X,Y] => Z): (X => Z, Y => Z) =
        ((x : X) =>  t2(Left(x : X) : Either[X, Y]) : Z,
         (y : Y) => t2(Right(y : Y) : Either[X, Y])  : Z)
    
    def to(t2: (X => Z, Y => Z)) : Either[X,Y] => Z = 
        e => e match{
            case Left(x) => t2._1(x) : Z
            case Right(y) => t2._2(y) : Z
        }
}


# $ \underline{·Tema \ 4: \ Isomorfismo \ de \ Curry-Howard:}$


$$ 
\begin{array}{cc}
\mathrm{\bf Scala\ ADTs} & \mathrm{\bf Logic} \\
\hline
\mathtt{Unit} & \top \ True \\
\mathtt{Nothing} & \bot  \ False\\
\mathtt{Either[P, Q]} & p \vee q \\
\mathtt{Tuple2[P, Q]} & p \wedge q \\
\mathtt{P => Q} & p \rightarrow q 
\end{array}
$$


```scala
((P => Q, P)) => Q
```

corresponds to the following formula of propositional logic: 

$\hspace{0.8cm} ((p->q) \land p)->p $

Let's get used to read Scala types as propositional formulas with these examples: 

$$
\begin{array}{cc}
\mathrm{\bf Scala\ types} & \mathrm{\bf Propositions} \\
\hline
\mathtt{(P, Either[Q, R])} & p \land (q \ \lor \ r)  \\ 
\mathtt{P => Q => (Q, R)} & p -> q \ -> q \land r \\
\mathtt{(P => Nothing) => Nothing} & (p -> false) -> false\\ 
\end{array}
$$

### ·Not y <=>:

In [ ]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P=>Q,Q=>P)

### ·$ \underline{Deducción \ Natural:}$

In [ ]:
def proof[P, Q, R](p1: Either[P, Q],
                     p2: P => Not[R],
                     p3: Q => Not[R]) : Not[R] =
    p1 match {
        case Left(p: P) => p2(p) : Not[R]
        case Right(q: Q) => p3(q) : Not[R]
    }